In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'object-detection-sample-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F802717%2F1376389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T112015Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D401a8fbed83c44520b1fa060d58243c9f5fc47c18b26a4754c5749d5e13908a8e0207102e24879c22ccc32c6b1be4c4913c98eb015d5391e68d7851a867462a85d5bd0cbd563d07d899358fc14009079d12f60b541e9a1b3ffc61b0a859b38bd0a0450dbcc3a9ebd3d1b8a3729786714d543ca75f8ac35c75e7920abaa3d0a54b5a14673c7d7d9427843b09baecf37b5971264a85093d563198e4d407c96c295e232a6df66f23443b11aa59b2e4ad5227e7be2eaf578239fc340ae8437f76e520ad50188ef63fbfe5fe2f17d16537a07a902efc607f1f45e5c0771d83125c9860ccac3297f08f6a9e0f771eb82b41c59cc509242bf3f537ddedd3e3eea32e69f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Object Detection with Pretrained TensorFlow Model - Tutorial

In this notebook, we'll learn about using a Pretrained Model from TensorFlow's Model Zoo to perform an Object Detection on a random Image. I have kept this notebook fairly organized and well-commented for easy reading. This notebook is specifically designed as a starter-code so that anyone who is interested in using Pretrained Model for Object Detection can simply fork it and start using it.

## Libraries

In [ ]:
# TensorFlow Libraries
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Generic
import os
import random

In [ ]:
# Enter a Random Images Directory Path
random_images = '../input/object-detection-sample-images/'

## Utility Functions - Display Image

In [ ]:
# Function to Display Image
def display_image(image,file):
    fig = plt.figure(figsize=(15, 10))
    plt.grid(False)
    plt.imshow(image)
    plt.title(file, fontsize=20)


# Function to Resize Image
def resize_image(img, new_width=300, new_height=300,display=False):
    img_path = os.path.join(random_images,img)
    pil_image = Image.open(img_path)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    if display:
        display_image(pil_image, img)

In [ ]:
# Applying the Function
resize_image(random.choice(os.listdir(random_images)),600,400,True)

## Utility Functions - Bounding Boxes

In [ ]:
# Function to Draw Bounding Boxes on Image
def draw_bounding_box_on_image(image,ymin,xmin,ymax,xmax,color,font,thickness=4,display_str_list=()):

    # Adds a bounding box to an image.
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),(left, top)],width=thickness,fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]

    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height

    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),(left + text_width, text_bottom)],fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),display_str, fill="black", font=font)
        text_bottom -= text_height - 2 * margin


# Function to Draw Boxes
def draw_boxes(image, boxes, class_names, scores, max_boxes=3, min_score=0.1):

    # Overlay labeled boxes on an image with formatted scores and label names.
    colors = list(ImageColor.colormap.values())
    font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")

            draw_bounding_box_on_image(image_pil,ymin,xmin,ymax,xmax,color,font,display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

## Pretrained Models

TensorFlow's Model Zoo has a couple of pretrained Models for Object Detection that are trained on COCO, KITTI & Open Images dataset. These Models can be used for inference if we're interested in categories from these datasets.

However, these pretrained Models are also useful for training on new datasets. The various architectures used in the pretrained model are described below:

* FasterRCNN + InceptionResNet v2 - Trained on Open Images dataset. It is slow but has high accuracy
* SSD + Mobilenet v2 - Trained on COCO dataset. It is small and quick

For this tutorial we shall be using using **FasterRCNN + InceptionResNet v2**

In [ ]:
# FasterRCNN + InceptionResNet v2 Path
model_path = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# Define Model
PreTrained_Model = hub.load(model_path).signatures['default']

## Utility Functions - Object Detection

In [ ]:
# Function to Load Image from Random Image Folder using TensorFlow
def load_img(img_file):
    path = os.path.join(random_images,img_file)
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [ ]:
# Function to Detect Object using TensorFlow's Pretrained Model
def detect(model, img_file):

    img = load_img(img_file)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    start_time = time.time()   # Start Timer

    result = model(converted_img)   # Detection Using Pretrained Model downloaded above

    end_time = time.time()    # End Timer

    result = {key:value.numpy() for key,value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time (secs): ",'{:.2}'.format(end_time-start_time))

    image_with_boxes = draw_boxes(img.numpy(), result["detection_boxes"],result["detection_class_entities"], result["detection_scores"])

    display_image(image_with_boxes,img_file)


In [ ]:
# Running the Detect Function
detect(PreTrained_Model, random.choice(os.listdir(random_images)))

Source for this Tutorial


[https://www.kaggle.com/code/scratchpad/notebook010dd3db7c/edit](https://)